## How to inference Hugging Face models using Langchain

In [25]:
# install the necessay packages
! pip -q install langchain langchain_community langchain_google_genai


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: C:\Users\HP\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [26]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [27]:
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# prompt template for evaluating answers
template = """Evaluate the following question and check whether the given answer is correct or not.

Question: {question}
Answer: {answer}
"""

# create a PromptTemplate instance with the defined template and input variables
prompt = PromptTemplate(
    template=template,
    input_variables=["question", "answer"]
)

# initialize the Hugging Face model endpoint for inference
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [28]:
# create a processing chain of the prompt and model
chain = prompt | model

question = "What is 2+2?"
answer = "5"

# invoke the chain with the question and answer, retrieving the result
result = chain.invoke({
    "question": f"{question}",
    "answer": f"{answer}"
})

In [29]:
print(result.content)

The given answer is **incorrect**.

*   **Question:** What is 2+2?
*   **Correct Answer:** 4
*   **Given Answer:** 5

2 + 2 equals 4, not 5.


## Langchain with Pydantic Schema

In [30]:
from typing import List, Dict, Optional # for complex outputs
from pydantic import Field, BaseModel
from langchain.output_parsers import PydanticOutputParser

In [31]:
class EvaluationSchema(BaseModel):
    # This sets the default value as the text provided.
    # correct: str = Field("Check if the provided answer is correct or not - TRUE or FALSE.")

    correct: bool = Field(description="Check if the provided answer is correct or not.")

In [32]:
class QuestionEvaluator:
    def __init__(self) -> None:

        # intialize the prompt template for evaluating questions and answers
        self.template = """
        Evaluate the following question and check whether the given answer is correct or not.

        Question: {question}
        Answer: {answer}

        {format_instructions}
        """

        # initialize the Hugging Face model endpoint for evaluation
        self.model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

        # initialize a parser for output
        self.parser = PydanticOutputParser(pydantic_object=EvaluationSchema)

    def get_chain(self):

        # retrieving format instructions from the parser
        format_instructions = self.parser.get_format_instructions()

        # creating a PromptTemplate instance with the evaluation template and input variables
        prompt = PromptTemplate(
            template=self.template,
            input_variables=[
                "question",
                "answer"
            ],
            partial_variables={
                "format_instructions": format_instructions
            }
        )

        # creating a processing chain that combines prompt, model, and parser
        chain = prompt | self.model | self.parser

        return chain

    def invoke_chain(self, question, answer):

        # load the processing chain
        chain = self.get_chain()

        # invoke the chain
        result = chain.invoke({
            "question": f"{question}",
            "answer": f"{answer}"
        })

        return result

In [33]:
question = "What is 2+2?"
answer = "5"

# create instance of QuestionEvaluator
evaluator = QuestionEvaluator()

# invoke the evaluation chain with the specified question and answer
result = evaluator.invoke_chain(
    question=question,
    answer=answer
).model_dump()

In [34]:
print(result)

{'correct': False}
